In [9]:
import numpy as np

In [10]:
class Tensor(object):
    def __init__(self, data, 
                autograd=False,
                creators=None,
                creation_op=None,
                id=None):
        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {}
        
        if(id is None):
            id = np.random.randint(0, 100000)
            
        self.id = id
        
        # 텐서가 얼마나 많은 자식을 가지고 있는지 계속해서 추적
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                    
                else:
                    c.children[self.id] += 1
                    
                    
    # 텐서가 각 자식으로부터 보낸 경사도 개수가 정확한지 확인                
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if(cnt != 0):
                return False
        return True
    
    def backward(self, grad=None, grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                # 역전파가 가능한지 또는 경사도를 기다리고 있는 상태인지를 확인하고, 경사도를 기다리고 있다면 카운터를 감소
                
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1
            
            # 여러 자식의 경사도를 누적       
            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
                
            if(self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    # 실제 역전파 시작
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data, autograd = True, creators=[self, other], creation_op="add")
        
        return Tensor(self.data + other.data)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())
    
    
        

In [11]:
class SGD(object):
    def __inint__(self, parameters, alpha=0.1):
        self.parametes = parameters
        self.alpha = alpha
        
    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0
            
    def step(self, zero
             =True):
        for p in self.parameters:
            p.data -= p.grad * self.alpha
            
            if(zero):
                p.grad.data *= 0

# 계층 형식 지원하기
### 상용 프레임워크가 지원하는 계층 형식을 우리 프레임워크에 추가해봅시다.

In [12]:
class Layer(object):
    def __init__(self):
        self.parameters = list()
    
    def get_parameters(self):
        return self.parameters

In [13]:
class Linear(Layer):
    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        W = np.random.randn(n_inputs, n_outputs)*np.sqrt(2.0/(n_inputs))
        self.weight = Tensor(W, autograd=True)
        self.bias = Tensor(np.zeros(n_outputs), autograd=True)
        
        self.parameters.append(self.weight)
        self.parameters.append(self.bias)
        
    def forward(self, input):
        return input.mm(self.weight)+self.bias.expand(0, len(input.data))
        
        

--------------------------

# 계층을 포함하는 계층
### 계층은 다른 계층을 포함할 수 있다.
* #### Sequential layer: 계층들의 리스트를 `순전파`하는 계층  

In [14]:
class Sequential(Layer):
    def __init__(self, layers=list()):
        super().__init__()
        self.layers = layers
        
    def add(self, layer):
        self.layers.append(layer)
        
    def forward(self, input):
        for layer in self.layers:
            input = layer.forward(input)
            
        return input
    
    def get_parameters(self):
        params = list()
        for l in self.layers:
            params += l.get_parameters()
            
        return params
    
    

In [15]:
data = Tensor(np.array([[0, 0],[0, 1], [1, 0], [1, 1]]), autograd=True)
target = Tensor(np.array([[0], [1], [0], [1]]), autograd=True)

model = Sequential([Linear(2, 3), Linear(3, 1)])
optim = SGD(parameters=model.get_parameters(), alpha=0.05)

for i in range(10):
    # 예측
    pred = model.forward(data)
    
    loss = ((pred-target)**2).sum(0)
    
    loss.backward(Tensor(np.ones_like(loss.data)))
    optim.step()
    print(loss)
    

TypeError: SGD() takes no arguments

----------------------------------------
# 손실 함수 계층
### 어떤 계층에는 가중치가 하나도 없습니다.

In [18]:
# MSE = Mean Squared Error (평균제곱오차)

class MSELoss(Layer):
    def __init__(self):
        super().__init__()
        
    def forward(self, pred, target):
        return ((pred - target)**2).sum(0)
    

In [19]:
np.random.seed(0)

In [20]:
data = Tensor(np.array([[0,0],[0,1],[1,0],[1,1]]), autograd=True)
target = Tensor(np.array([[0],[1],[0],[1]]), autograd=True)

In [21]:
model = Sequential([Linear(2, 3), Linear(3, 1)])
criterion = MSELoss()
optim = SGD(parameters=model.get_parameters(), alpha=0.05)

TypeError: SGD() takes no arguments

In [22]:
for i in range(10):
    pred = model.forward(data)
    
    loss = criterion.forward(pred, target)
    
    loss.backward(Tensor(np.ones_like(loss.data)))
    optim.step()
    print(loss)

AttributeError: 'Tensor' object has no attribute 'mm'